In [ ]:
# Loading execution and data client configuration
from nautilus_trader.adapters.interactive_brokers.config import InteractiveBrokersDataClientConfig
from nautilus_trader.adapters.interactive_brokers.config import InteractiveBrokersExecClientConfig

# Loading instrument provider configuration
from nautilus_trader.adapters.interactive_brokers.config import InteractiveBrokersInstrumentProviderConfig
from nautilus_trader.adapters.interactive_brokers.config import SymbologyMethod

# Historical data client
from nautilus_trader.adapters.interactive_brokers.historical.client import HistoricInteractiveBrokersClient
from ibapi.common import MarketDataTypeEnum

# Retrieving contracts
from nautilus_trader.adapters.interactive_brokers.common import IBContract

In [ ]:
# Example for TWS paper trading (default port 7497)
data_config = InteractiveBrokersDataClientConfig(
    ibg_host="127.0.0.1",
    ibg_port=7497,
    ibg_client_id=1,
)

exec_config = InteractiveBrokersExecClientConfig(
    ibg_host="127.0.0.1",
    ibg_port=7497,
    ibg_client_id=1,
    account_id="DUN137629",  # Your paper trading account ID
)

In [ ]:
instrument_provider_config = InteractiveBrokersInstrumentProviderConfig(
    symbology_method=SymbologyMethod.IB_SIMPLIFIED,
    load_ids=frozenset([
        #"EUR/USD.IDEALPRO",    # Forex
        "MSFT.NASDAQ",           # Stock
        #"BTC/USD.PAXOS",       # Crypto
    ]),
)

In [ ]:
instrument_provider_config

# Fetching historical data

In [ ]:
# Initialize the client
client = HistoricInteractiveBrokersClient(
    host="127.0.0.1",
    port=7497,
    client_id=1,
    market_data_type=MarketDataTypeEnum.DELAYED_FROZEN,  # Use delayed data if no subscription
    log_level="INFO"
)

# Connect to TWS/Gateway
await client.connect()

# Fetching contract information

In [ ]:
# Define contracts
contracts = [
    #IBContract(secType="STK", symbol="AAPL", exchange="SMART", primaryExchange="NASDAQ"),
    IBContract(secType="STK", symbol="MSFT", exchange="SMART", primaryExchange="NASDAQ"),
    #IBContract(secType="CASH", symbol="EUR", currency="USD", exchange="IDEALPRO"),
]

# Request instrument definitions
instruments = await client.request_instruments(contracts=contracts)

In [ ]:
instruments[0]

In [ ]:
import datetime

# Request historical bars
bars = await client.request_bars(
    bar_specifications=[
        "1-MINUTE-LAST",    # 1-minute bars using last price
        "5-MINUTE-MID",     # 5-minute bars using midpoint
        "1-DAY-LAST",       # Daily bars using last price
    ],
    start_date_time=datetime.datetime(2026, 1, 23, 9, 30),
    end_date_time=datetime.datetime(2026, 1, 23, 10, 00),
    tz_name="America/New_York",
    contracts=contracts,
    use_rth=True,  # Regular Trading Hours only
    timeout=120,   # Request timeout in seconds
)

In [ ]:
bars

In [ ]:
import asyncio 
import datetime 
from nautilus_trader.adapters.interactive_brokers.common import IBContract 
from nautilus_trader.adapters.interactive_brokers.historical.client import HistoricInteractiveBrokersClient 
from nautilus_trader.persistence.catalog import ParquetDataCatalog 

async def download_historical_data(): 
    # Initialize client 
    client = HistoricInteractiveBrokersClient( host="127.0.0.1", port=7497, client_id=5, ) 
    
    # Connect
    await client.connect() 
    await asyncio.sleep(2) # Allow connection to stabilize 
    
    # Define contracts 
    contracts = [ 
        IBContract(secType="STK", symbol="AAPL", exchange="SMART", primaryExchange="NASDAQ"), 
        IBContract(secType="CASH", symbol="EUR", currency="USD", exchange="IDEALPRO"), 
    ] 
    
    # Request instruments 
    instruments = await client.request_instruments(contracts=contracts) 
    
    # Request historical bars     
    bars = await client.request_bars( 
        bar_specifications=["1-HOUR-LAST", "1-DAY-LAST"], 
        start_date_time=datetime.datetime(2023, 11, 1, 9, 30), 
        end_date_time=datetime.datetime(2023, 11, 6, 16, 30), 
        tz_name="America/New_York", 
        contracts=contracts, 
        use_rth=True, 
    ) 
    
    # Request tick data 
    ticks = await client.request_ticks( 
        tick_types=["TRADES"], 
        start_date_time=datetime.datetime(2023, 11, 6, 14, 0), 
        end_date_time=datetime.datetime(2023, 11, 6, 15, 0), 
        tz_name="America/New_York", 
        contracts=contracts, 
    ) 
    
    # Save to catalog 
    #catalog = ParquetDataCatalog("./catalog") 
    #catalog.write_data(instruments) 
    #catalog.write_data(bars) 
    #catalog.write_data(ticks) 
    
    print(f"Downloaded {len(instruments)} instruments") 
    print(f"Downloaded {len(bars)} bars") 
    print(f"Downloaded {len(ticks)} ticks") 
    
    # Disconnect 
    await client.disconnect() 
    
# Run the example 
if __name__ == "__main__": 
    asyncio.run(download_historical_data())